In [ ]:
import pandas as pd
import tensorflow as tf
import tensorflow_hub as hub
import numpy as np
import spacy
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from sklearn.preprocessing import OneHotEncoder
import os
os.environ["TOKENIZERS_PARALLELISM"] = "true"

nlp = spacy.load('en_core_web_lg')

In [ ]:
# df.to_pickle("./pickle_files/df_spojeni_vectors.pkl")
# df = pd.read_pickle("./pickle_files/df_spojeni_vectors.pkl")
# df.to_pickle("./pickle_files/df_odvojeni_vectors.pkl")
# df = pd.read_pickle("./pickle_files/df_odvojeni_vectors.pkl")
# df.to_pickle("./pickle_files/df_odvojeni_word_vectors.pkl")
df = pd.read_pickle("./pickle_files/df_odvojeni_word_vectors.pkl")
# df = df.head(300000)

In [ ]:
#spajanje odvojenih postova

df = df.groupby(['type', 'avgWordLen'], sort=False,as_index=False).agg({'posts':' '.join,'upperCount':'first','stopWordCount':'first','urlCount':'first','wordCount':'first','avgWordLen':'first','vectors':lambda x:np.array(x)})

In [ ]:
# df0 = df[df['type'] == 0].sample(450)
# df1 = df[df['type'] == 1].sample(450)
# df2 = df[df['type'] == 2].sample(450)
# df3 = df[df['type'] == 3].sample(450)

# df = pd.concat([df0, df1, df2, df3],)
# df = df.reset_index().drop('index', axis=1)

In [ ]:
#padding vektora
def paddingEmbedded(a):
  b = np.zeros([len(a), len(max(a, key=lambda x: len(x))),300])
  for i, j in enumerate(a):
    if(len(j)==0):
      continue
    b[i][0:len(j)] = j
    b[i][len(j):] = np.zeros(300,)
  return b


df['vectors'] = df['vectors'].apply(lambda x: np.array(list(filter(lambda y: y.ndim > 1, x))))
df = df[df.vectors.map(lambda d: len(d)) >0].reset_index().drop('index',axis=1)
df['vectors'] = df['vectors'].apply(lambda x: np.array(list(map(lambda y: y.reshape(300,), np.concatenate(x)))))
vectorList = paddingEmbedded(df['vectors'].array)

## NN ALGORITHMS

In [ ]:
training_size = 0.8

sentences = []
labels = []

for ind in df.index:
    sentences.append(vectorList[ind])
    labels.append(df['type'][ind])

In [ ]:
from sklearn.model_selection import train_test_split
training_sentences, testing_sentences, training_labels, testing_labels = train_test_split(sentences, labels, test_size=1-training_size, random_state=42, stratify=labels)

In [ ]:
# Need this block to get it to work with TensorFlow 2.x
import numpy as np
training_labels = np.array(training_labels)
testing_labels = np.array(testing_labels)

In [ ]:
encoder = OneHotEncoder()
training_labels = encoder.fit_transform(training_labels.reshape(-1,1)).toarray()
testing_labels = encoder.fit_transform(testing_labels.reshape(-1,1)).toarray()

In [ ]:
# testing_sentences = np.array(list(testing_sentences))
# training_sentences = np.array(list(training_sentences))
testing_sentences = tf.convert_to_tensor(testing_sentences)
training_sentences = tf.convert_to_tensor(training_sentences)

In [ ]:
classes = 4

filters = 32
pool_size = 2
kernel_size = 3

model = tf.keras.Sequential([
    tf.keras.layers.Dropout(0.8),
    tf.keras.layers.Conv1D(filters, kernel_size, padding='valid',activation='relu'),
    tf.keras.layers.MaxPooling1D(pool_size),
    tf.keras.layers.LSTM(120,return_sequences=True),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(classes,activation='softmax')
])
model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),optimizer=tf.keras.optimizers.Adam(),metrics=['accuracy'])

In [ ]:
NEpoch = 32
batch_size = 32
history = model.fit(training_sentences, training_labels, batch_size=batch_size, epochs=NEpoch, validation_data=(testing_sentences, testing_labels))

In [ ]:
model.save('saved_models/450Set_spacy')
model.save('saved_models/450Set_spacy.h5')


In [ ]:
import matplotlib.pyplot as plt


def plot_graphs(history, string):
  plt.plot(history.history[string])
  plt.plot(history.history['val_'+string])
  plt.xlabel("Epochs")
  plt.ylabel(string)
  plt.legend([string, 'val_'+string])
  plt.show()
  
plot_graphs(history, "accuracy")
plot_graphs(history, "loss")

In [ ]:
def paddingEmbedded(a):
  b = np.zeros([len(a), len(max(a, key=lambda x: len(x))), 300])
  for i, j in enumerate(a):
    if (len(j) == 0):
      continue
    b[i][0:len(j)] = j
    b[i][len(j):] = np.zeros(300,)
  return b

def word2vec(post):
  vecList = []
  # post = list(map(lambda x: x.split(),post))
  docs = nlp(post)
  for doc in docs:
    vecList.append(doc.vector)

  vecList=np.array(vecList)
  return np.array(vecList)



sentence = ["I fully believe power protector voice voiceless so spirit I present film hope recieve spirit compassion om mani padme hum yes right but time help come outside relationshippartner supportive helpful I think get right help critical finding good if do not feel worthy love difference I think jawz I think indicative deep issue long meet it mean friend lookingglass thank make point unfortunately corrupt christianity speak loud voice still small voice forget very christian today this curse I get to tell people prefer remain stuck neurosis excuse of course spite scar I ve I know painful callie honestly I think dodged bullet I m sure he s good person sound emotional mature being involve you guy singe song I ve research heuristic mental shortcuts cognitive bias explain lot people for principle I empathize sentiment I think truly religion contain need society easily manipulate control this dualistic I think forget type make person emotional social competency past hurt issue value meme etc I see experiment crowd I do not feel rare I feel thank uk sound like reasonable compassionate person I think neitzsche quote beware fight monster lest process appropriate here colorado it surprise learn cognitive brain science discover human actually hard - wire empathy read golemans social intelligence help understand you understand aegis blame yes like inhuman monster try justify marginalize human people like do not understand you know idea riddle fallacy linquistic manipulation is not worth time continue conversation hopefully year pass learn think an elephant swallow boa constrictor love book I cry little end thoughtful philosophy sure I like line the little prince signature block I use accord logic tolerance accept intolerance consider tolerant homosexuality violent behavior conflate extremely okay guy record I think healthy debate help forward simple heuristic accompany cognitive bias prevalent religion critical thinking look like explain I m sorry I do not notice post do not mean ignore the teleological argument definitely major flaw conclusion draw okay attribute else quote second problem this post logical fallacy introduce straw - man no problem look light moment re - frame argument no introduce idea tolerant clever use slippery slope do not blame touch gay marriage good I far faith reason inherently contradictory I word post way I you understand I spend life study religion no suppose know inner council can not people aware communicate telepathically year disguise regular folk so get decide good and I would not word like logic religion concern little religion stand critical inquiry old school infj theme song veteran psychic war blue oyster cult okay world bullshit I m go to change frame reference up point I ve operate think world vampire define get olderfor life I try squeeze role I think I need order I think people know like you plate pardner pretty typical ill bet stuff go inside reasonably articulate this I love I like card do not write creative after argument wife I find little dog I write run away love thread over year I practice religion philosophy I think infj mean explorer extent during phase exploration I practice I think I hate crap meaningless usually people feel good the truth people self - center do not understand love I understand it bother lot I walk store feel angry judgmental add sap energy feeling I do not like the great sage linus van pelt say I love mankind people I can not stand I find end individually I love people group funny come long way sometimes I try talk like oh man do not know sure chance come accord cognitive brain science truly tune human synchrony brain wave with discovery mirror neuron look like stumble buddhist first noble truth okay I m go to sound lame go you go find keep change grow nature ever try friend stray cat the rule apply do not chase run provide safe friendly harbor watch quickly affectionate well like someones post thank un thank thank oh chazz that feeling come go I would not mean ever I remember I feel like I belong as child I impression presence okay totally agree post I could not help get visual intimidate typist I see roz monsters inc it take little time tell difference anxious perception clear intuition that say year I manage screw royally time great movie I think movie move wing of desire this film old school style direct wim wender feature talent I m sure experience maybe I m middle aged I thing way I musician artist salesman soldier nurse cna technician christian buddhist I find mirror neuron read daniel golemans social intelligence credit research go italian research scientist escape ai not funive reading mirror neuron genuinely engage human brain actually sync now have compile information link mb type soma type I wonder expectation little unrealistic time sometimes I feel emotionally raw wound need withdraw oddly time like little",
            "I really enjoyed this anime, but some of the characters were a bit annoying. Also, it would have been a lot better if the ending wasn't so horrible.",
            "From time to time, I have the urge to punch a random stranger on the street. My intrusive thoughts are hard to control, and one time, when I couldn't win in a fight against them, I ended up in prison. I still don't remember what i did. The rumor has it that I killed them."]
embedded = []
for post in sentence:
  embedded.append(word2vec(post))

embedded = np.array(embedded)

embedded = paddingEmbedded(embedded)

print(model.predict(embedded))
## correct is
## [2]
## [0]

In [ ]:
print(max(history.history['val_accuracy']))
print(min(history.history['val_loss']))


# Evaluation

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
import seaborn as sns


In [ ]:
predict = model.predict(testing_sentences)

In [ ]:
from math import isnan
################ CONFUSION MATRIX ####################
# Get and reshape confusion matrix data
matrix = confusion_matrix(np.argmax(predict,axis=1), np.argmax(testing_labels,axis=1))
matrix = matrix.astype('float') / matrix.sum(axis=1)[:, np.newaxis]


for index1, i in enumerate(matrix):
  for index2, j in enumerate(i):
    if isnan(j):
      matrix[index1][index2] = 0;

# Build the plot
plt.figure(figsize=(16, 7))
sns.set(font_scale=1.4)
sns.heatmap(matrix, annot=True, annot_kws={'size': 10},
            cmap=plt.cm.Greens, linewidths=0.2)

# Add labels to the plot
class_names = ['GUARDIAN', 'ARTISAN', 'IDEALIST', 'RATIONALIST']
tick_marks = np.arange(len(class_names))
tick_marks2 = tick_marks + 0.5
plt.xticks(tick_marks, class_names, rotation=25)
plt.yticks(tick_marks2, class_names, rotation=0)
plt.xlabel('Predicted label')
plt.ylabel('True label')
plt.title('Confusion Matrix for Random Forest Model')
plt.show()
################ CONFUSION MATRIX ####################


In [ ]:
import sklearn.metrics as skm

pred_max = np.argmax(predict,axis=1)
test_max = np.argmax(testing_labels,axis=1)

print(skm.classification_report(test_max,pred_max))